**Installing Caffe Lirary (CPU or GPU)**

In [ ]:
#Caffe Installation:

#CPU
!apt install caffe-cpu

#GPU
# !apt install caffe-gpu

**Required Packages Importing**

In [2]:
#Packages Importing:
import numpy as np
import lmdb
import caffe

**From Numpy Array to LMDB**

In [15]:
#Write Function:
def writeArray(file_name, Data, Label, index):
  """Creates LMDB Folder from for a given folder.

  # Arguments
      file_name: string name for LMDB folder to be created. 
      Data: data to be converted in Numpy Array format.
      Labels: label for the data in Numpy Array format.
      index: the index to be assigned for the data in LMDB file.

  # Returns
      True: if LMDB folder was created
      None: otherwise
  """

  X = Data.copy()
  y = Label.copy()

  map_size = X.nbytes * 10
  channels = X.shape[0]
  height = X.shape[1]
  width =  X.shape[2]
  idx = index

  env = lmdb.open(file_name, map_size=map_size)

  with env.begin(write=True) as txn:
    # txn is a Transaction object
    datum = caffe.proto.caffe_pb2.Datum()
    datum.channels = channels
    datum.height = height
    datum.width = width
    datum.data = X.tobytes()  # or .tostring() if numpy < 1.9
    datum.label = int(y)

    str_id = '{:08}'.format(idx)
    print(str_id)

    # The encode is only essential in Python 3
    txn.put(str_id.encode('ascii'), datum.SerializeToString())

    return True

**From LMDB to Numpy Array**

In [16]:
#Read Function:
def readArray(file_name, index_range = (0,0)):
  """Reads LMDB files from in a given folder and converts them to Numpy Arrays.

  # Arguments
      file_name: string name for LMDB folder contains LMDB files to be converted t Numpy Arrays format.
      index_range: tuple contains range of data to be parsed to Numpy Array in LMDB file.

  # Returns
      x: Numpy Array for the resulted data.
      y: Numpy Array for the resulted labels.
  """

  cursor_len = 0
  x, y = [], []
  lower = index_range[0]
  upper = index_range[1]

  env = lmdb.open(file_name, readonly=True)
  with env.begin() as txn:
    cursor = txn.cursor()
    cursor_len = len(list(cursor))

    for i in range(lower, upper+1):
      idx = '{:08}'.format(i).encode('ascii')
      raw_datum = txn.get(idx)
      datum = caffe.proto.caffe_pb2.Datum()
      datum.ParseFromString(raw_datum)
      # print(datum.channels, datum.height, datum.width)
      
      flat_x = np.frombuffer(datum.data, dtype=np.uint8)
      # flat_x = np.array(bytearray(datum.data))
      # flat_x = np.fromstring(datum.data, dtype=np.uint8)
      x.append(flat_x.reshape(datum.channels, datum.height, datum.width))
      y.append(datum.label)

  x, y = np.asarray(x), np.asarray(y)

  return x, y

**Return LMDB File keys**

In [5]:
#LMDB keys function
def lmdbKeys(file_name):
  """Reads LMDB file keys.

  # Arguments
      file_name: string name for LMDB folder contains LMDB files to return back the keys from.

  # Returns
      keys: list of LMDB file keys.
  """
  env = lmdb.open(file_name, readonly=True)
  keys = []
  with env.begin() as txn:
    cursor = txn.cursor()
    print(crusor)
    for key, value in cursor:
      keys.append(key)

  return keys

*Below loop for looping and reading the images, to be modified to match the "create_image_lists" function*

In [35]:
# Write PNG
print('Reading PNG (LMDB)\n\n')

import cv2
import glob

imPNG = glob.glob('/content/PokemonPNG/*.png')
file_namePNG = 'TEST_LMDB_PNG'
for idx, img in enumerate(imPNG):
  i = cv2.imread(img)
  i = np.asarray(i)
  print(i.shape)
  writeArray(file_namePNG, i, np.asarray([idx]), idx)

Reading PNG (LMDB)


(400, 400, 3)
00000000
(400, 400, 3)
00000001
(400, 400, 3)
00000002


In [34]:
#Write JPG
print('Writing JPG (LMDB)\n\n')

import cv2
import glob

imJPG = glob.glob('/content/PokemonJPG/*.jpg')
file_nameJPG = 'TEST_LMDB_JPG'
for idx, img in enumerate(imJPG):
  i = cv2.imread(img)
  i = np.asarray(i)
  print(i.shape)
  writeArray(file_nameJPG, i, np.asarray([idx]), idx)

Writing JPG (LMDB)


(400, 400, 3)
00000000
(400, 400, 3)
00000001
(400, 400, 3)
00000002


In [24]:
# Read PNG

file_namePNG = '/content/TEST_LMDB_PNG'
xPNG, yPNG = readArray(file_namePNG, index_range = (0,2))

In [32]:
print('Reading PNG (LMDB)\n\n')

for im in xPNG:
  print(im.shape)
  # cv2_imshow(im)
  # cv2_imshow(im[:,:,:3])

Reading PNG (LMDB)


(400, 400, 3)
(400, 400, 3)
(400, 400, 3)


In [22]:
# Read JPG

file_nameJPG = '/content/TEST_LMDB_JPG'
xJPG, yJPG = readArray(file_nameJPG, index_range = (0,2))

In [33]:
print('Reading JPG (LMDB)\n\n')

for im in xJPG:
  print(im.shape)
  # cv2_imshow(im)
  # cv2_imshow(im[:,:,:3])

Reading JPG (LMDB)


(400, 400, 3)
(400, 400, 3)
(400, 400, 3)


**Reading PNG and JPG images (imread) w & w/o extra channel**

In [30]:
print('Reading PNG (imread)\n\n')

img1 = cv2.imread('/content/PokemonPNG/001.png')
print('Reading without (-1) results with 3 Dim:::')
print('cv2.imread(\'/content/PokemonPNG/001.png\')>>>>>>>>>>>', img1.shape, '\n')

img2 = cv2.imread('/content/PokemonPNG/001.png', -1)
print('Reading with (-1) results with 4 Dim:::')
print('cv2.imread(\'/content/PokemonPNG/001.png, -1\')>>>>>>>', img2.shape)

Reading PNG (imread)


Reading without (-1) results with 3 Dim:::
cv2.imread('/content/PokemonPNG/001.png')>>>>>>>>>>> (400, 400, 3) 

Reading with (-1) results with 4 Dim:::
cv2.imread('/content/PokemonPNG/001.png, -1')>>>>>>> (400, 400, 4)


In [31]:
print('Reading JPG (imread)\n\n')

img3 = cv2.imread('/content/PokemonJPG/001.jpg')
print('Reading without (-1) results with 3 Dim:::')
print('cv2.imread(\'/content/PokemonJPG/001.jpg\')>>>>>>>>>>>', img3.shape, '\n')

img4 = cv2.imread('/content/PokemonJPG/001.jpg', -1)
print('Reading with (-1) results with 3 Dim:::')
print('cv2.imread(\'/content/PokemonJPG/001.jpg, -1\')>>>>>>>', img4.shape)

Reading JPG (imread)


Reading without (-1) results with 3 Dim:::
cv2.imread('/content/PokemonJPG/001.jpg')>>>>>>>>>>> (400, 400, 3) 

Reading with (-1) results with 3 Dim:::
cv2.imread('/content/PokemonJPG/001.jpg, -1')>>>>>>> (400, 400, 3)
